In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

PROJECT_ROOT = '/content/drive/MyDrive/Colab Notebooks/soai'   # 변경 시 여기만 수정
import sys, os
sys.path.append(PROJECT_ROOT)                  # agent.py 임포트를 위해

Mounted at /content/drive


In [2]:
!pip install -qU langchain langchain-community \
                langchain-google-genai \
                sentence-transformers transformers accelerate
!pip install -qU langgraph

In [3]:
# from google.colab import userdata

# os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
# os.environ["LANGCHAIN_PROJECT"] = "fantasy-npc-project"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [4]:
# (A) Gemini – 가장 단순
from langchain_google_genai import ChatGoogleGenerativeAI
import os, torch
from google.colab import userdata

api_key = userdata.get('GEMINI_API_KEY')
gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             google_api_key=api_key,
                             temperature=0.8)

# (B) Hermes-3 8B (GPU 여유 있을 때)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
model_id = "NousResearch/Hermes-3-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    # load_in_4bit=True,  # 메모리 절약을 위한 4비트 양자화
    trust_remote_code=True
)
# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    temperature=0.9,
    do_sample=True,
    return_full_text=False,      # 새로 추가: 입력 프롬프트 제외하고 생성된 텍스트만 반환
    pad_token_id=tokenizer.eos_token_id,
    # repetition_penalty=1.1       # 새로 추가: 반복 방지
)
hermes = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
<ipython-input-4-3036507717>:35: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hermes = HuggingFacePipeline(pipeline=pipe)


In [5]:
# import importlib, sys

# agent.py 파일 수정한 경우 아래 코드 실행하기
# 수정한 내용을 반영하려면 ↓ 두 줄이면 충분합니다.
# importlib.invalidate_caches()          # 파이썬 모듈 캐시 초기화
# if "agent" in sys.modules:             # 이미 로드돼 있었다면
#     importlib.reload(sys.modules["agent"])
# else:
#     import agent                       # 처음 불러오는 경우


In [8]:
from agent import build_langgraph          # agent.py 안의 함수

graph1 = build_langgraph(
    file_path   = os.path.join(PROJECT_ROOT, "asuna"),
    analysis_llm=gemini,
    response_llm=gemini,
    summary_llm =gemini,
    analysis_name="gemini",
    response_name="gemini",
    summary_name="gemini",
    me="asuna",
    you="rocky"
)

graph2 = build_langgraph(
    file_path   = os.path.join(PROJECT_ROOT, "rocky"),
    analysis_llm=gemini,
    response_llm=gemini,
    summary_llm =hermes,
    analysis_name="gemini",
    response_name="gemini",
    summary_name="hermes",
    me="rocky",
    you="asuna"
)

In [7]:
import time

state = {"question": "Let's check out today's quest! Which do you think good for us?"}

for i in range(20):
    asuna_ans = graph1.invoke(state)
    print("asuna: ", asuna_ans["response"])
    state = {"question": asuna_ans["response"]}
    rocky_ans = graph2.invoke(state)
    print("rocky: ", rocky_ans["response"])
    state = {"question": rocky_ans["response"]}
    time.sleep(20)

asuna:  Given our current levels and equipment, I'd recommend prioritizing the quest in the Whispering Woods that involves clearing out the corrupted treants. It offers a substantial experience reward and the potential to acquire rare crafting materials, which would be more beneficial in the long run than the goblin extermination quest. Plus, facing a challenge that requires a bit more strategy would serve as better training for our upcoming raid. I'm also curious about your thoughts on improving our party's coordination; perhaps we could discuss strategies after the quest?
rocky:  --- Response ---
Asuna, I believe tackling the corrupted treant quest is a better choice. The rewards are more substantial and the experience will be beneficial for leveling up our characters. Plus, dealing with a treant would be a good challenge for us to work on our party coordination and strategies. After completing the quest, we could analyze our performance and discuss potential improvements, like commu

KeyboardInterrupt: 